In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob
import scipy.stats as stats

#matplot 한글폰트 설정
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\GULIM.TTC").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"]= False


## 6번 농장 J/Day 결측치 처리
### 2. Imputation of missing values 방법들로 결측값 대체

In [31]:
tom_path = "C:/seokwonna/pythonwork/AI/FinalProject/tomato_farm_hour_data"
tom6 = pd.read_csv(tom_path + "/farm6_hour.csv", index_col=0, encoding='euc-kr')

In [35]:
tom6_copy = tom6
tom6_copy

,날짜,J/Day,평균온도,EC급액,EC배액,pH급액,pH배액,내부이산화탄소,온도토양,온도급액,온도배액
0,2021-10-25 11:00:00,1552.0,25.113070,2.830347,3.722351,5.947333,6.743198,346.968750,10.627604,14.991536,12.888672
1,2021-10-25 12:00:00,1552.0,24.407475,2.805600,3.761688,5.919826,6.748368,343.335484,21.581048,25.834677,24.141532
2,2021-10-25 13:00:00,1552.0,24.842023,2.721864,3.812324,5.923776,6.651351,342.298507,22.169776,25.094216,24.510261
3,2021-10-25 14:00:00,1552.0,24.607697,2.749988,3.906761,5.920353,6.653017,347.647059,22.758929,25.038866,24.942227
4,2021-10-25 15:00:00,1552.0,22.910198,2.713705,3.848237,5.928817,6.644458,361.525000,22.901563,24.300521,24.143229
...,...,...,...,...,...,...,...,...,...,...,...
1365,2021-12-21 08:00:00,1186.0,13.585806,2.446037,3.035981,5.622514,5.860288,900.972973,14.860360,14.969032,16.012387
1366,2021-12-21 09:00:00,1186.0,17.653590,2.474195,3.109794,5.743450,5.867016,813.751938,15.259205,15.635659,17.381783
1367,2021-12-21 10:00:00,1186.0,22.535974,2.639204,3.334455,5.682939,5.962366,592.099237,16.438454,18.394561,19.302958
1368,2021-12-21 11:00:00,1186.0,23.122648,2.731169,3.276885,5.512000,6.034342,470.508333,17.441146,20.093229,19.914583


In [34]:
tom6_copy.describe()

,J/Day,평균온도,EC급액,EC배액,pH급액,pH배액,내부이산화탄소,온도토양,온도급액,온도배액
count,1160.000000,441.000000,441.000000,441.000000,441.000000,441.000000,441.000000,441.000000,441.000000,441.000000
mean,1318.563856,17.483419,2.485320,3.205465,5.933521,6.238342,644.230696,17.195126,17.309818,17.346033
std,335.612472,3.517173,0.169044,0.249110,0.210164,0.279038,171.793641,7.832770,7.767336,7.809785
min,362.000000,12.066213,2.229501,2.861326,5.495000,5.798194,328.966387,-126.250000,-127.267857,-127.375000
25%,1113.500000,14.524529,2.342666,3.017070,5.699700,6.066500,485.392308,16.265752,15.643209,16.078381
50%,1369.000000,16.203120,2.453180,3.127694,5.997099,6.159810,653.805556,17.686170,16.975962,17.103070
75%,1551.325000,20.295256,2.591161,3.352955,6.079696,6.353163,797.267717,19.206771,19.819853,19.829464
max,1884.000000,25.349298,2.968739,3.946891,6.449359,6.982790,949.775000,23.170898,25.834677,25.032227


##### 토마토  column feature 양육 기준 정보
## - Reference 1: https://samwha27.com/entry/%EC%8A%A4%EB%A7%88%ED%8A%B8%ED%8C%9C-%EA%B8%B0%EC%B4%88%EA%B5%90%EC%9C%A1-%EC%96%91%EC%88%98%EB%B6%84-%EA%B4%80%EB%A6%AC%EB%A5%BC-%EB%B0%B0%EC%9B%8C%EB%B3%B4%EC%9E%90
## - Reference 2: https://smartfarm.rda.go.kr/dtsv/grwhct_pub.do
- J/Day : 600 ~ 1300
- 평균온도 :  17.4 ~    15 ~ 30도로 유지해야함
- EC 급액 : 2.5 ~ 3.5
- EC 배액 : 3.5 ~ 5.0
- pH 급액 : 5.2 ~ 5.9 or  ## 6.0 ~ 6.5 가 적절하다고 여김 
- pH 배액 : 5.0 ~ 7.0
- 이산화탄소농도 : 350 ~ 600
- 온도토양 : 15 ~ 25
- 온도급액 : 15 ~ 30
- 온도배액 : 15 ~ 30


In [ ]:
##  날짜 index set 

tomato_farm_list = ["tom6_copy"]
for farm in tomato_farm_list:
    farm_data = locals()[farm]
    farm_data.set_index('날짜', inplace=True)  # Set '날짜' column as the index

In [ ]:
# 결측치 시각화
# cmd에서 pip 설치 하세요 code : pip install missingno
import missingno as msno
msno.matrix(df=tom6_copy.iloc[:, :], color=(0.1, 0.6, 0.8))


In [ ]:
## bar chart
msno.bar(df=tom6_copy.iloc[:, :], color=(0.1, 0.6, 0.8))


In [ ]:
missing = tom6_copy.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(figsize = (12,6))

In [ ]:
tom6_copy = pd.DataFrame(tom6_copy)

In [ ]:
tom6_copy.info()

In [ ]:
tom6_copy.columns

In [ ]:
tom6_copy.columns

In [ ]:
# boxplot 이상치 확인
plt.figure(figsize = (6, 8))
for i in range(len(tom6_copy.columns)):
    plt.subplot(5, 3, i+1)
    sns.boxplot(tom6_copy[tom6_copy.columns[i]])
    plt.title(tom6_copy.columns[i])
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))
columnlist = ['J/Day', '평균온도', 'EC급액', 'EC배액', 'pH급액', 'pH배액', '내부이산화탄소', '온도토양', '온도급액', '온도배액']

for i, column in enumerate(columnlist):
    # Create a boxplot for the current column
    axes[i//4, i % 4].boxplot(tom6_copy[column])
    axes[i//4, i % 4].set_title(column)  # Set the title as the column name

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

tomato_farm_list = ["tom6_copy"]

iterative_imputed_data_list = []

for farm in tomato_farm_list:
    farm_data = locals()[farm]
    print("============================================================================")
    print(farm, "농장 데이터 정보:")
    print(farm_data.info())
    print("============================================================================")
    imputer = IterativeImputer(random_state=0)
    imputed_data = imputer.fit_transform(farm_data)
    imputed_df = pd.DataFrame(imputed_data, columns=farm_data.columns)
    iterative_imputed_data_list.append(imputed_df)
    locals()[farm + "_imputed"] = imputed_df

# 각 농장별 보간된 데이터프레임 출력
for i, farm in enumerate(tomato_farm_list):
    imputed_df = iterative_imputed_data_list[i]
    print("============================================================================")
    print(farm, "농장 보간된 데이터:")
    print(imputed_df)
    print("============================================================================")


In [ ]:
imputed_dataframe_count = len(iterative_imputed_data_list)
imputed_dataframe_names = [farm + "_iterative_imputed" for farm in tomato_farm_list]
print("보간된 데이터프레임 개수:", imputed_dataframe_count)
print("보간된 데이터프레임 변수명:", imputed_dataframe_names)

tom6_iterative_imputed = iterative_imputed_data_list[0]

In [ ]:
## 분석 시각화하기
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))

for i, (column1, column2) in enumerate(zip(tom6_copy.columns, tom6_iterative_imputed.columns)):
    ax = axes[i // 4, i % 4]
    tom6_copy[column1].plot(kind='kde', ax=ax, color='blue', label='Original')
    tom6_iterative_imputed[column2].plot(
        kind='kde', ax=ax, color='red', label='Iterative')
    ax.set_title(column1)
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
## simpleimputer
# 결측치 처리 - mean
from sklearn.impute import SimpleImputer
SI_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
SI_mean.fit(tom6_copy)
tom6_mean = SI_mean.transform(tom6_copy)
tom6_mean = pd.DataFrame(tom6_mean, columns=tom6_copy.columns)
tom6_mean


In [ ]:
# 결측치 처리 - median
from sklearn.impute import SimpleImputer
SI_median = SimpleImputer(missing_values=np.nan, strategy='median')
SI_median.fit(tom6_copy)
tom6_median = SI_median.transform(tom6_copy)
tom6_median = pd.DataFrame(tom6_median, columns=tom6_copy.columns)
tom6_median


In [ ]:
# 결측치 처리 - most_frequent
from sklearn.impute import SimpleImputer
SI_most_frequent = SimpleImputer(
    missing_values=np.nan, strategy='most_frequent')
SI_most_frequent.fit(tom6_copy)
tom6_most_frequent = SI_most_frequent.transform(tom6_copy)
tom6_most_frequent = pd.DataFrame(
    tom6_most_frequent, columns=tom6_copy.columns)
tom6_most_frequent


## 3)결측치 처리: KNNImputer

In [ ]:
from sklearn.impute import KNNImputer
imputer3 = KNNImputer(n_neighbors=3)
mm_knn3_tom6 = imputer3.fit_transform(tom6_copy)
mm_knn3_tom6 = pd.DataFrame(
    mm_knn3_tom6, columns=tom6_copy.columns)

imputer4 = KNNImputer(n_neighbors=4)
mm_knn4_tom6 = imputer4.fit_transform(tom6_copy)
mm_knn4_tom6 = pd.DataFrame(
    mm_knn4_tom6, columns=tom6_copy.columns)

imputer5 = KNNImputer(n_neighbors=5)
mm_knn5_tom6 = imputer5.fit_transform(tom6_copy)
mm_knn5_tom6 = pd.DataFrame(
    mm_knn5_tom6, columns=tom6_copy.columns)

imputer6 = KNNImputer(n_neighbors=6)
mm_knn6_tom6 = imputer6.fit_transform(tom6_copy)
mm_knn6_tom6 = pd.DataFrame(
    mm_knn6_tom6, columns=tom6_copy.columns)

imputer7 = KNNImputer(n_neighbors=7)
mm_knn7_tom6 = imputer7.fit_transform(tom6_copy)
mm_knn7_tom6 = pd.DataFrame(
    mm_knn7_tom6, columns=tom6_copy.columns)


In [ ]:
### 시각화 정리 -1 결측치 다변량, mean, median, most_frequent 결측치 처리
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))

for i, (column1, column2, column3, column4, column5) in enumerate(zip(tom6_copy.columns, tom6_iterative_imputed.columns,
                                                                      tom6_mean.columns, tom6_median.columns,
                                                                      tom6_most_frequent.columns)):
    ax = axes[i // 4, i % 4]
    tom6_copy[column1].plot(kind='kde', ax=ax, color='blue', label='Original')
    tom6_iterative_imputed[column2].plot(kind='kde', ax=ax, color='red',
                         label='다변량 배치')
    tom6_mean[column3].plot(
        kind='kde', ax=ax, color='green', label='Mean')
    tom6_median[column4].plot(
        kind='kde', ax=ax, color='orange', label='Median')
    tom6_most_frequent[column5].plot(
        kind='kde', ax=ax, color='purple', label='Most Frequent')
    ax.set_title(column1)
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
### 시각화 정리 -2 KNN, n_neighbor = 3, 4, 5, 6, 7
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))

for i, (column1, column2, column3, column4, column5, column6) in enumerate(zip(tom6_copy.columns, mm_knn3_tom6.columns,
                                                             mm_knn4_tom6.columns, mm_knn5_tom6.columns,
                                                             mm_knn6_tom6.columns, mm_knn7_tom6.columns)):
    ax = axes[i // 4, i % 4]
    tom6_copy[column1].plot(kind='kde', ax=ax, color='blue', label='Original')
    mm_knn3_tom6[column2].plot(kind='kde', ax=ax, color='red',
                             label='KNN_n3')
    mm_knn4_tom6[column3].plot(
        kind='kde', ax=ax, color='green', label='KNN_n4')
    mm_knn5_tom6[column4].plot(
        kind='kde', ax=ax, color='orange', label='KNN_n5')
    mm_knn6_tom6[column5].plot(
        kind='kde', ax=ax, color='yellow', label='KNN_n6')
    mm_knn7_tom6[column6].plot(
        kind='kde', ax=ax, color='aqua', label='KNN_n7')
    ax.set_title(column1)
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
"""
KNN_n 결측치 처리 모델 중, KNN_n_6, KNN_n_7 결측치 처리 모델이 적합하다고 판단되어 KNN_6 모델을 KNN_n 결측치 처리 모델로 사용하여 결측치 처리 모델 시각화에 다른 모델들과 비교
"""

In [ ]:
###cmd에서 pip 설치 하세요 code : pip install impyute
from impyute.imputation.cs import mice
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
tom6_copy

In [ ]:
## MICE(Multivariate Imputation by Chained Equation) package
np_imputed=mice(tom6_copy.values) # mice 학습시작
tom6_MICE = pd.DataFrame(np_imputed, columns = tom6_copy.columns)

# 변환 후 데이터 확인
tom6_MICE.head(10)

In [ ]:
### 시각화 정리 -1 결측치 다변량, mean, median,  결측치 처리
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))

for i, (column1, column2, column3, column4, column5, column6) in \
enumerate(zip(tom6_copy.columns, tom6_iterative_imputed.columns,
              tom6_mean.columns, tom6_median.columns,
              tom6_MICE.columns, mm_knn6_tom6.columns)):
    ax = axes[i // 4, i % 4]
    tom6_copy[column1].plot(kind='kde', ax=ax, color='blue', label='Original')
    tom6_iterative_imputed[column2].plot(kind='kde', ax=ax, color='red',
                         label='다변량 배치')
    tom6_mean[column3].plot(
        kind='kde', ax=ax, color='aqua', label='Mean')
    tom6_median[column4].plot(
        kind='kde', ax=ax, color='darkgray', label='Median')
    tom6_MICE[column5].plot(
        kind='kde', ax=ax, color='purple', label='MICE package')
    mm_knn6_tom6[column6].plot(
        kind='kde', ax=ax, color='lime', label='KNN_n6')
    
    ax.set_title(column1)
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
### 시각화 정리 final
# tom6_iterative_imputed, MICE_package, knn_n6 결측치 처리
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))

for i, (column1, column2, column3, column4, column5) in \
enumerate(zip(tom6_copy.columns, tom6_iterative_imputed.columns,
              tom6_mean.columns, tom6_MICE.columns, mm_knn6_tom6.columns)):
    ax = axes[i // 4, i % 4]
    tom6_copy[column1].plot(kind='kde', ax=ax, color='blue', label='Original')
    tom6_iterative_imputed[column2].plot(kind='kde', ax=ax, color='red',
                         label='iterative')
    tom6_mean[column3].plot(kind='kde', ax=ax, color='violet',
                                         label='mean')
    tom6_MICE[column4].plot(kind='kde', ax=ax, color='cyan',
                         label='MICE')
    mm_knn6_tom6[column5].plot(
        kind='kde', ax=ax, color='olive', label='KNN_n6')
    
    ax.set_title(column1)
    ax.legend()

plt.tight_layout()
plt.show()


## 이상치 확인 및 제거하기
### Dataframe set 은 tom6_iterative_imputed, MICE_package, knn_n6 결측치 처리 data frame을 각각 처리 할 예정

In [ ]:
# tom6_copy.columns, tom6_iterative_imputed.columns,
#               tom6_mean.columns, tom6_MICE.columns, mm_knn6_tom6.columns

In [ ]:
## 중간 save
# C:/seokwonna/pythonwork/AI/FinalProject/결측치처리_20230627

tom6_iterative_imputed.to_csv(
    "C:/seokwonna/pythonwork/AI/FinalProject/결측치처리_20230627/tom6_iterative_imputed.csv", encoding='euc-kr')
tom6_mean.to_csv(
    "C:/seokwonna/pythonwork/AI/FinalProject/결측치처리_20230627/tom6_mean.csv", encoding='euc-kr')
tom6_MICE.to_csv(
    "C:/seokwonna/pythonwork/AI/FinalProject/결측치처리_20230627/tom6_MICE.csv", encoding='euc-kr')
mm_knn6_tom6.to_csv("C:/seokwonna/pythonwork/AI/FinalProject/결측치처리_20230627/mm_knn6_tom6.csv", encoding='euc-kr')

In [ ]:
# matplotlib
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rcParams['axes.unicode_minus'] = False  # matplotlib 마이너스기호 표시
plt.rc('font', family='NanumGothic')  # matplotlib 한글폰트 표시



In [ ]:
tom6_iterative_imputed.columns
columnlist = ['J/Day', '평균온도', 'EC급액', 'EC배액', 'pH급액', 'pH배액', '내부이산화탄소', '온도토양',
              '온도급액', '온도배액']
tom6_iterative_imputed.boxplot(column=['J/Day'])


In [ ]:
# tom6_iterative_imputed dataframe columns outlier check
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))
columnlist = ['J/Day', '평균온도', 'EC급액', 'EC배액',
              'pH급액', 'pH배액', '내부이산화탄소', '온도토양', '온도급액', '온도배액']

for i, column in enumerate(columnlist):
    # Create a boxplot for the current column
    axes[i//4, i % 4].boxplot(tom6_iterative_imputed[column])
    axes[i//4, i % 4].set_title(column)  # Set the title as the column name

plt.tight_layout()
plt.show()


In [ ]:
tom6_iterative_imputed.sort_values(by='온도토양', ascending=True)


In [ ]:
# tom6_mean dataframe columns outlier check
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))
columnlist = ['J/Day', '평균온도', 'EC급액', 'EC배액',
              'pH급액', 'pH배액', '내부이산화탄소', '온도토양', '온도급액', '온도배액']

for i, column in enumerate(columnlist):
    # Create a boxplot for the current column
    axes[i//4, i % 4].boxplot(tom6_mean[column])
    axes[i//4, i % 4].set_title(column)  # Set the title as the column name

plt.tight_layout()
plt.show()


In [ ]:
# tom6_MICE dataframe columns outlier check
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))
columnlist = ['J/Day', '평균온도', 'EC급액', 'EC배액',
              'pH급액', 'pH배액', '내부이산화탄소', '온도토양', '온도급액', '온도배액']

for i, column in enumerate(columnlist):
    # Create a boxplot for the current column
    axes[i//4, i % 4].boxplot(tom6_MICE[column])
    axes[i//4, i % 4].set_title(column)  # Set the title as the column name

plt.tight_layout()
plt.show()


In [ ]:
# mm_knn6_tom6 dataframe columns outlier check
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(18, 12))
columnlist = ['J/Day', '평균온도', 'EC급액', 'EC배액',
              'pH급액', 'pH배액', '내부이산화탄소', '온도토양', '온도급액', '온도배액']

for i, column in enumerate(columnlist):
    # Create a boxplot for the current column
    axes[i//4, i % 4].boxplot(mm_knn6_tom6[column])
    axes[i//4, i % 4].set_title(column)  # Set the title as the column name

plt.tight_layout()
plt.show()
